In [ ]:
import xml.sax

In [2]:
filename = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'

In [ ]:
class RegexExp():
    def __init__(self):
        pass
    def textHandling(self,ID,text,title):
        text = text.lower()
        data = text.split('==references==')
        if len(data) == 1:
            data = text.split('==references==')
        if len(data) == 1:
            references = []
            links = []
            categories = []

In [3]:
class XMLHandler(xml.sax.ContentHandler):
    def __init__(self):
        self.CurrentData = ''
        self.title = ''
        self.ID = ''
        self.idFlag = 0
        self.text = ""
        
    def startElement(self,tag,attributes):
        self.CurrentData = tag
            
    def endElement(self,tag):
        
        if self.CurrentData == 'title':
            print(self.title)
        elif self.CurrentData == 'text':
            print(self.text)
        self.CurrentData = ''
        self.title = ''
        self.ID = ''
        self.idFlag = 0
        self.text = ''
        
    def characters(self,content):
        if self.CurrentData == 'title':
            self.title += content
        elif self.CurrentData == 'text':
            self.text += content
        elif self.CurrentData == 'id' and self.idFlag = 0:
            self.ID = content
            self.idFlag = 1

In [ ]:
class Parser():
    def __init__(self,filename):
        self.parser = xml.sax.make_parser()
        self.parser.setFeature(xml.sax.handler.feature_namespaces, 0)
        self.handler =  XMLHandler()
        self.parser.setContentHandler(self.handler)
        self.parser.parse(filename)

In [4]:
parser = Parser(filename)

AccessibleComputing
{{R unprintworthy}}
Anarchism
[[Category:Social theories]]
AfghanistanHistory
}}
AfghanistanGeography
}}
AfghanistanPeople
}}
AfghanistanCommunications
}}
AfghanistanTransportations
}}
AfghanistanMilitary
}}
AfghanistanTransnationalIssues
}}
AssistiveTechnology
}}
AmoeboidTaxa
}}
Autism
[[Category:Words coined in the 1910s]]
AlbaniaHistory
}}
AlbaniaPeople
}}
AsWeMayThink
}}
AlbaniaGovernment
}}
AlbaniaEconomy
}}
Albedo
[[Category:Words coined in the 1760s]]
AfroAsiaticLanguages
}}
ArtificalLanguages
}}
AbacuS
}}
AbalonE
}}
AbbadideS
}}
AbbesS
}}
AbbevilleFrance
}}
AbbeY
}}
AbboT
}}
Abbreviations
}}
AtlasShrugged
}}
ArtificialLanguages
}}
AtlasShruggedCharacters
}}
AtlasShruggedCompanies
}}
AyersMusicPublishingCompany
}}
AfricanAmericanPeople
}}
AdolfHitler
}}
AbeceDarians
}}
AbeL
}}
AbensbergGermany
}}
AberdeenSouthDakota
}}
ArthurKoestler
}}
AynRand
}}
AlexanderTheGreat
}}
AnchorageAlaska
}}
ArgumentForms
}}
ArgumentsForTheExistenceOfGod
}}
AnarchY
}}
AsciiArt
}}
